In [ ]:
import numpy as np
import torch

def lambdaReturns(rewards, values, gamma=0.99, lambda_=0.95):
    n = len(rewards)
    returns = torch.zeros(n)
    bootstrap = values[-1]
    for t in reversed(range(n)):
        returns[t] = rewards[t] + gamma*((1 - lambda_)*values[t + 1] + lambda_*bootstrap)
        bootstrap = returns[t]
    return returns

rewards = [1, 0, 0, 1, 0, 2]
values = [0.5, 0.6, 0.4, 0.7, 0.9, 0.8, 0.0]

gamma = 0.99
lambda_ = 0.95
lmbdaReturns = lambdaReturns(rewards, values, gamma, lambda_)

print("Rewards:", rewards)
print("Values:", values)
print("Lambda-returns:", lmbdaReturns)

Rewards: [1, 0, 0, 1, 0, 2]
Values: [0.5, 0.6, 0.4, 0.7, 0.9, 0.8, 0.0]
Lambda-returns: tensor([3.4506, 2.5741, 2.7159, 2.8509, 1.9206, 2.0000])


In [25]:
rewards = torch.tensor([1, 0, 0, 1, 0, 2])
values = torch.tensor([0.5, 0.6, 0.4, 0.7, 0.9, 0.8])
rewards = rewards[:-1]
next_values = values[1:]
last = next_values[-1]
horizonLength = 6
gamma = 0.99
inputs = rewards + gamma * next_values * (1 - lambda_)

outputs = []
# single step
for index in reversed(range(horizonLength - 1)):
    last = inputs[index] + gamma * lambda_ * last
    outputs.append(last)
returns = torch.stack(list(reversed(outputs)))
print("Lambda-returns:", returns)

Lambda-returns: tensor([2.5676, 1.6352, 1.7176, 1.7894, 0.7920])


In [24]:
import numpy as np
import torch

def lambdaReturns(rewards, values, gamma=0.99, lambda_=0.95):
    horizonLength = len(rewards)
    bootstrap = values[-1]
    returns = torch.zeros(horizonLength - 1)
    for t in reversed(range(horizonLength - 1)):
        returns[t] = rewards[t] + gamma*((1 - lambda_)*values[t + 1] + lambda_*bootstrap)
        bootstrap = returns[t]
    return returns

rewards = [1, 0, 0, 1, 0, 2]
values = [0.5, 0.6, 0.4, 0.7, 0.9, 0.8]

gamma = 0.99
lambda_ = 0.95
lmbdaReturns = lambdaReturns(rewards, values, gamma, lambda_)

print("Rewards:", rewards)
print("Values:", values)
print("Lambda-returns:", lmbdaReturns)

Rewards: [1, 0, 0, 1, 0, 2]
Values: [0.5, 0.6, 0.4, 0.7, 0.9, 0.8]
Lambda-returns: tensor([2.5676, 1.6352, 1.7176, 1.7894, 0.7920])


In [20]:
import torch

def lambdaReturns(rewards, nextValues, lambda_=0.95, gamma=0.99):
    td_targets = rewards + gamma * nextValues
    returns = torch.zeros_like(rewards)
    bootstrap = td_targets[-1]  # Initialize with last TD target
    for t in reversed(range(len(rewards))):
        returns[t] = (1 - lambda_) * td_targets[t] + lambda_ * bootstrap
        bootstrap = rewards[t] + gamma * returns[t]
    return returns

# Example usage
rewards = torch.tensor([1., 0., 0., 1., 0., 2.])
values = torch.tensor([0.5, 0.6, 0.4, 0.7, 0.9, 0.8])

# Use all but last reward, and all values
lambda_returns = lambdaReturns(rewards[:-1], values[1:])
print("Lambda returns:", lambda_returns)

Lambda returns: tensor([1.6676, 1.6884, 1.7741, 0.8394, 0.7920])


In [23]:
rewards = torch.tensor([1., 0., 0., 1., 0., 2.])
values = torch.tensor([0.5, 0.6, 0.4, 0.7, 0.9, 0.8])
horizonLength = len(rewards)
returns = torch.zeros_like(rewards[:-1])

bootstrap = values[-1]
for i in reversed(range(len(returns))):
    returns[i] = rewards[i] + gamma * ((1 - lambda_)*values[i] + lambda_*bootstrap)
    bootstrap = returns[i]

print(f"returns: {returns}")

returns: tensor([2.5545, 1.6265, 1.6978, 1.7842, 0.7970])
